# Importando as bibliotecas





In [1]:
import tensorflow as tf
tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
tf.config.list_physical_devices('GPU')


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2640625569948878519
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2252026676
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7828232581623758661
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
gpus = tf.config.list_logical_devices('GPU')

type(gpus)
if gpus:
  # Replicate your computation on multiple GPUs
  c = []
  for gpu in gpus:
    with tf.device(gpu.name):
        print(gpu.name)
        pass
gpu = gpus[0]

/device:GPU:0


In [4]:
import sys
import os
from sklearn.model_selection import train_test_split
sys.path.append('./')
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import ThresholdedReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import models
import h5py
from tensorflow.keras.models import Model
from keras import Input
from tensorflow.keras import optimizers
import json
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import sklearn.metrics as sklm
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
#from attention_decoder import AttentionDecoder

In [5]:
class Data(object):
    """
    Class to handle loading and processing of raw datasets.
    """
    def __init__(self, data_source,
                 alphabet=" abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}",
                 input_size=1014, num_of_classes=3, alphabet_size=71):
        """
        Initialization of a Data object.

        Args:
            data_source (str): Raw data file path
            alphabet (str): Alphabet of characters to index
            input_size (int): Size of input features
            num_of_classes (int): Number of classes in data
        """
        self.alphabet = alphabet
        self.alphabet_size = len(self.alphabet)
        self.dict = {}  # Maps each character to an integer
        self.no_of_classes = num_of_classes
        for idx, char in enumerate(self.alphabet):
            self.dict[char] = idx + 1
        self.length = input_size
        self.data_source = data_source

    def load_data(self):
        """
        Load raw data from the source file into data variable.

        Returns: None

        """
        data = []
        for doc in self.data_source:
            data.append(doc)
        self.data = np.array(data)

    def get_all_data(self):
        """
        Return all loaded data from data variable.

        Returns:
            (np.ndarray) Data transformed from raw to indexed form with associated one-hot label.

        """
        data_size = len(self.data)
        start_index = 0
        end_index = data_size
        batch_texts = self.data[start_index:end_index]
        batch_indices = []
        one_hot = np.eye(self.no_of_classes, dtype='int64')
        classes = []
        for c, s in batch_texts:
            batch_indices.append(self.str_to_indexes(s))
            c = int(c) - 1
            classes.append(one_hot[c])
        return np.asarray(batch_indices, dtype='int64'), np.asarray(classes)

    def str_to_indexes(self, s):
        """
        Convert a string to character indexes based on character dictionary.
        
        Args:
            s (str): String to be converted to indexes

        Returns:
            str2idx (np.ndarray): Indexes of characters in s

        """
        s = s.lower()
        max_length = min(len(s), self.length)
        str2idx = np.zeros(self.length, dtype='int64')
        for i in range(1, max_length + 1):
            c = s[-i]
            if c in self.dict:
                str2idx[i - 1] = self.dict[c]
        return str2idx

    def vectorize_sentences(self, data, char_indices):
        X = []

        for doc in data:
        
            x = [char_indices[w] if w in self.alphabet else 0 for w in doc.lower()]
            x2 = np.eye(self.alphabet_size + 1)[x]
            X.append(x2)
            
        return pad_sequences(X, maxlen=self.length, padding="pre")

    def get_target_data(self):
        """
        Return all loaded data from data variable.

        Returns:
            (np.ndarray) Data transformed from raw to indexed form with associated one-hot label.

        """
        data_size = len(self.data)
        start_index = 0
        end_index = data_size
        batch_texts = self.data[start_index:end_index]
        batch_indices = []
        one_hot = np.eye(self.no_of_classes, dtype='int64')
        classes = []
        for c, s in batch_texts:
            c = int(c)
            classes.append(one_hot[c])
        return np.asarray(classes)

    def get_data_3dim(self):
        '''
        transform training input to shape Number_of_Rows X MaxLen X Alphabet_size
        this is need to be done to avoid the use of embedding layer, wich is not supported on the Tflite converter
        '''
        char_indices = dict((c, i) for i, c in enumerate(self.alphabet,1))
        indices_char = dict((i, c) for i, c in enumerate(self.alphabet,1))

        
        train_data = self.vectorize_sentences(self.data, char_indices)
        train_data.shape
        return train_data


    

In [6]:
CHARNN_DATA_CONFIG = "D:/SiDi/Project/Modulo II/github/multimodal-sentiment-analysis/rbsa2/charnn_cnn/charcnn_data_config.json"
charcnn_data_config = pd.read_json(CHARNN_DATA_CONFIG)

CNN_DATA_CONFIG = "D:/SiDi/Project/Modulo II/github/multimodal-sentiment-analysis/rbsa2/charnn_cnn/nn_config.json"
nn_data_config = pd.read_json(CNN_DATA_CONFIG)


In [7]:
#data_config = json.load(open("charcnn_data_config.json")) #Leitura do arquivo de configuracao
data_config = charcnn_data_config #Leitura do arquivo de configuracao

#nn_config = json.load(open("nn_config.json")) # Leitura de arquivo de configuracao, quais sao os parametros
nn_config = nn_data_config # Leitura de arquivo de configuracao, quais sao os parametros

alphabet_size = data_config["data"]["alphabet_size"]+1 #tamanho do alfabeto
input_size = data_config["data"]["input_size"] #Tamanho do input de zi

fully_connected_layers = nn_config["char_cnn_zhang"]["fully_connected_layers"] #Camadas Conectadas
conv_layers = nn_config["char_cnn_zhang"]["conv_layers"] #Camadas Convulacionasi
threshold = nn_config["char_cnn_zhang"]["threshold"] # Limiar
dropout_p = nn_config["char_cnn_zhang"]["dropout_p"] #Taxa de Dropout
embedding_size = nn_config["char_cnn_zhang"]["embedding_size"] # Tamnho do Embedding

In [8]:
DATASET_PATH_URL  = "D:/SiDi/Project/Modulo II/dataset/dataset_sidi_512.csv"
df = pd.read_csv(DATASET_PATH_URL, sep='\t')

print(len(df))
df.reset_index(inplace=True)

12000


In [9]:
# Verifica o balanceamento das classes
classes = df.groupby('sent_text')['index'].nunique()/df.shape[0] #Agrupe por index
classes

sent_text
NEG    0.333333
NEU    0.333333
POS    0.333333
Name: index, dtype: float64

In [10]:

#Funcao que remove quebra de linha
def remove_spaces(cell):  
    return cell.replace('\n',' ')

#Mapeaa os tipos de classes existente nos rotulos de predicoes
def to_one_hot(y, class_mapping, num_classes):
    y_int = y.map(class_mapping)
    return to_categorical(y_int, num_classes)

#X_text = df['text'].apply(remove_spaces)


# Numero de classes do dataset

In [11]:

num_classes = df['sent_text'].nunique()
#Faz o mapeeamento da classe com o id
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['sent_text']))}
class_mapping
#y_int = df['sent_text'].map(class_mapping)

{'NEG': 0, 'NEU': 1, 'POS': 2}

# Agrupamento e Separação por Rotulo da Classe (Sentimento da Texto)

In [12]:
SLICING_SIZE = 1000
neg_df = df.groupby(['sent_text']).get_group("NEG").sample(n=SLICING_SIZE, ignore_index=True)
neu_df = df.groupby(['sent_text']).get_group("NEU").sample(n=SLICING_SIZE, ignore_index=True)

pos_df = df.groupby(['sent_text']).get_group("POS").sample(n=SLICING_SIZE, ignore_index=True)

## 1D Text Datraframe

In [13]:
X_join_text = []
y_join = []

text_path = []
y_total = []

timesteps = 3
y_train_lf = []
X_lf_pp_train = []
y_true_train_docs = []


for i in range(0,SLICING_SIZE, 1):
    # Negative Class
    X_text = neg_df.iloc[i].to_frame().T['text']
    y = to_one_hot(neg_df.iloc[i].to_frame().T['sent_text'], class_mapping, num_classes)
    X_join_text.extend(X_text)
    y_join.extend(y)

    # Positive Class
    X_text = neu_df.iloc[i].to_frame().T['text']
    y = to_one_hot(neu_df.iloc[i].to_frame().T['sent_text'], class_mapping, num_classes)
    X_join_text.extend(X_text)
    y_join.extend(y)
    
    # Neutro Class
    X_text = pos_df.iloc[i].to_frame().T['text']
    y = to_one_hot(pos_df.iloc[i].to_frame().T['sent_text'], class_mapping, num_classes)
    X_join_text.extend(X_text)
    y_join.extend(y)







In [14]:
np.unique(y_join,  axis=0, return_counts=True)

(array([[0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.]], dtype=float32),
 array([1000, 1000, 1000], dtype=int64))

In [15]:
len(y_join)

3000

In [16]:
#https://stackoverflow.com/questions/50997928/typeerror-only-integer-scalar-arrays-can-be-converted-to-a-scalar-index-with-1d#answer-50997969
st =  StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1234)
for train_index, test_index in st.split(X_join_text, y_join):
    print(train_index)
    X_train_text, X_test_text = pd.Series(X_join_text)[train_index.astype(int)],pd.Series(X_join_text)[test_index.astype(int)]
    
    y_train, y_test = np.array(y_join)[train_index.astype(int)], np.array(y_join)[test_index.astype(int)]
    print(len(train_index), len(test_index))



[2492   30 2981 ...   71 2305 1559]
2400 600


# Transforma Data

In [17]:
training_data = Data(data_source=X_train_text,
                         alphabet=data_config["data"]["alphabet"],
                         input_size=data_config["data"]["input_size"],
                         num_of_classes=data_config["data"]["num_of_classes"],
                         alphabet_size=data_config["data"]["alphabet_size"])
training_data.load_data()

validation_data = Data(data_source=X_test_text,
                         alphabet=data_config["data"]["alphabet"],
                         input_size=data_config["data"]["input_size"],
                         num_of_classes=data_config["data"]["num_of_classes"],
                         alphabet_size=data_config["data"]["alphabet_size"])
validation_data.load_data()
    

X_train_text = training_data.get_data_3dim()
X_test_text = validation_data.get_data_3dim()

In [18]:
np.unique(y_train, axis=0, return_counts=True)

(array([[0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.]], dtype=float32),
 array([800, 800, 800], dtype=int64))

In [19]:
np.unique(y_test, axis=0, return_counts=True)

(array([[0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.]], dtype=float32),
 array([200, 200, 200], dtype=int64))

# Conjunto de Dados de Validacao textValidationCheckpoint

In [20]:
def predict(model, X_test_text, batch_size=32):
    y_predict = np.round(model.predict(X_test_text))
    return y_predict

def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

class TextValidationCheckpoint(Callback):
    def __init__(self, filepath, X_test_text,y_test, metric = 'kappa'):
        self.X_test_text = X_test_text
        self.y_test = y_test
        self.metric = metric
        self.max_metric = float('-inf')
        self.max_metrics = None
        self.filepath = filepath
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        
        predicted_labels = predict(self.model, self.X_test_text)
        true_labels = one_hot_decode(self.y_test)
        predicted_labels = one_hot_decode(predicted_labels)
        eval_metrics = {
            'accuracy' : sklm.accuracy_score(true_labels, predicted_labels),
            'f1_micro' : sklm.f1_score(true_labels, predicted_labels, average='micro'),
            'f1_macro' : sklm.f1_score(true_labels, predicted_labels, average='macro'),
            #'f1_binary' : sklm.f1_score(true_labels, predicted_labels, average='weighted', pos_label = 1),
            'kappa' : sklm.cohen_kappa_score(true_labels, predicted_labels)
        }
        eval_metric = eval_metrics[self.metric]
        self.history.append(eval_metrics)
        
        if epoch > -1 and eval_metric > self.max_metric:
            print("\n" + self.metric + " improvement: " + str(eval_metric) + " (before: " + str(self.max_metric) + "), saving to " + self.filepath)
            self.max_metric = eval_metric     # optimization target
            self.max_metrics = eval_metrics   # all metrics
            #self.model.save(self.filepath)





# Create Architecture

In [21]:


text_input = Input(shape=(input_size, alphabet_size), name='text_input', dtype='float32')

x = Convolution1D(activation='tanh',
                     input_shape=(input_size, alphabet_size), filters=50, kernel_size=1)(text_input)

for cl in conv_layers:
    x = Convolution1D(cl[0], cl[1])(x)
    #x = ThresholdedReLU(threshold)(x)
    if cl[2] != -1:
        x = MaxPooling1D(cl[2])(x)
x = Flatten()(x)

for fl in fully_connected_layers:
    x = Dense(fl)(x)
    #x = ThresholdedReLU(threshold)(x)
    x = Dropout(dropout_p)(x)

encoded_text = Dense(256, activation='tanh', name='encoded_text')(x)

output = layers.Dense(num_classes, activation='softmax', name='output')(encoded_text)
model_text = Model(text_input, output)
model_text.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-4),
                    metrics=['acc'])
print(model_text.summary())


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 2880, 102)]       0         
                                                                 
 conv1d (Conv1D)             (None, 2880, 50)          5150      
                                                                 
 conv1d_1 (Conv1D)           (None, 2874, 64)          22464     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 958, 64)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 952, 64)           28736     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 317, 64)          0         
 1D)                                                         

C:\Users\rbsa2\multimodal-sentiment-analysis\multimodal-sentiment-analysis\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


# Train and Validate Results

In [22]:
n_repeats = 1
exp_history = []
optimize_for = 'accuracy'
model_file = 'txt_tweet_text_v1_test.h5'
for i in range(n_repeats):
    checkpoint = TextValidationCheckpoint(model_file, X_test_text, y_test, metric=optimize_for)
    model_text.fit(X_train_text, y_train, epochs=26, batch_size=16,
                     validation_data=(X_test_text, y_test)
                     ,callbacks = [checkpoint])
    exp_history.append(checkpoint.max_metrics)
    


Epoch 1/26
150/150 [==============================] - ETA: 0s - loss: 1.0966 - acc: 0.3688
accuracy improvement: 0.3333333333333333 (before: -inf), saving to txt_tweet_text_v1_test.h5
150/150 [==============================] - 22s 107ms/step - loss: 1.0966 - acc: 0.3688 - val_loss: 1.1303 - val_acc: 0.3450
Epoch 2/26
150/150 [==============================] - 15s 98ms/step - loss: 1.0821 - acc: 0.3887 - val_loss: 1.0886 - val_acc: 0.3783
Epoch 3/26
150/150 [==============================] - ETA: 0s - loss: 1.0319 - acc: 0.4700
accuracy improvement: 0.4116666666666667 (before: 0.3333333333333333), saving to txt_tweet_text_v1_test.h5
150/150 [==============================] - 15s 99ms/step - loss: 1.0319 - acc: 0.4700 - val_loss: 1.0059 - val_acc: 0.5083
Epoch 4/26
150/150 [==============================] - ETA: 0s - loss: 0.9923 - acc: 0.4917
accuracy improvement: 0.4716666666666667 (before: 0.4116666666666667), saving to txt_tweet_text_v1_test.h5
150/150 [==============================

# Print the Results

In [23]:
avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp_history]) / n_repeats
print(avg_result)
print(avg_acc)
for i, r in enumerate(exp_history):
    model_file = "ness_exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)


0.4125
0.6083333333333333
0 0.4125 0.6083333333333333 0.6081345723789299 ness_exp1_single-page_repeat-00.hdf5


# Export Result

In [27]:
import pickle
pickle.dump( checkpoint.history, open( "bt4sa-text-tweet-sent-multi-cnn.pickle", "wb" ) )


# Export Model

In [24]:
model_text.save('v0.0.1_bt4sa-text-tweet-sent.h5')

# Predict

In [25]:
import time
start = time.time()
y_pred = model_text.predict(X_test_text)

print(time.time()-start)

1.299215316772461


#  Classification Metric Result

In [26]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index

pred_class = []
for i in range(len(y_pred)):
  pred = np.where(y_pred[i] == np.amax(y_pred[i]))
  pred_class.append(pred[0][0])
    
print(classification_report(Y_test, pred_class))

              precision    recall  f1-score   support

           0       0.59      0.66      0.62       200
           1       0.62      0.52      0.56       200
           2       0.60      0.64      0.62       200

    accuracy                           0.60       600
   macro avg       0.60      0.60      0.60       600
weighted avg       0.60      0.60      0.60       600

